In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [ ]:
CFG = {
    "csv_path":        "mumbai_weekly_tariffs.csv",  # Mumbai CSV you supplied
    "battery_kwh":     13.5,          # usable energy
    "battery_kw":      5.0,           # ± continuous power
    "eta_round":       0.95,          # round-trip efficiency
    "soc_initial":     0.3,           # 0-1
    "house_load_kw":   0.8,           # kW
    "plot_hours":      168,           # horizon to visualise
}

In [ ]:
def read_mumbai_csv(path: str) -> pd.DataFrame:
    """Return hourly DataFrame with Asia/Kolkata DatetimeIndex."""
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["Date"])
    df["start_hour"] = df["Time_Slot"].str.split("-").str[0].str[:2].astype(int)
    df["timestamp"] = df["date"] + pd.to_timedelta(df["start_hour"], unit="h")

    hourly = []
    for _, r in df.iterrows():
        idx = pd.date_range(r["timestamp"], freq="H", periods=6, tz="Asia/Kolkata")
        hourly.append(pd.DataFrame({"timestamp": idx, "tariff": r["Tariff_(₹/kWh)"]}))
    dfh = pd.concat(hourly).drop_duplicates("timestamp").sort_values("timestamp")
    return dfh.set_index("timestamp")

In [ ]:
def plan_hour(tariff: float, load_kw: float,
              soc_prev: float, bat_kwh: float, bat_kw: float, eta: float) -> dict:
    """
    Returns {
        grid_kw: float,      # >0 import
        battery_kw: float,   # + discharge, – charge
        soc_new:  float,     # 0–1
        mode:     str        # "charge" | "discharge"
    }
    """
    if tariff >= 6.5:  # high-price → discharge
        max_discharge_energy = min(bat_kw, soc_prev * bat_kwh)
        discharge_kw = min(max_discharge_energy, load_kw / eta)
        grid_kw = max(0.0, load_kw - discharge_kw * eta)
        battery_kw = discharge_kw
        soc_new = soc_prev - discharge_kw / bat_kwh
        mode = "discharge"
    else:  # low-price → charge
        max_charge_energy = min(bat_kw, (1 - soc_prev) * bat_kwh)
        battery_kw = -max_charge_energy
        grid_kw = load_kw + max_charge_energy
        soc_new = soc_prev + max_charge_energy * eta / bat_kwh
        mode = "charge"

    return dict(grid_kw=grid_kw,
                battery_kw=battery_kw,
                soc_new=np.clip(soc_new, 0, 1),
                mode=mode)

In [ ]:
def simulate(df_future: pd.DataFrame, cfg: dict) -> pd.DataFrame:
    """df_future must have DatetimeIndex and tariff column."""
    records = []
    soc = cfg["soc_initial"]
    for ts, row in df_future.iterrows():
        res = plan_hour(row["tariff"], cfg["house_load_kw"],
                        soc, cfg["battery_kwh"], cfg["battery_kw"], cfg["eta_round"])
        records.append({
            "timestamp" : ts,
            "tariff"    : row["tariff"],
            "battery_kw": res["battery_kw"],
            "grid_kw"   : res["grid_kw"],
            "soc"       : res["soc_new"],
            "mode"      : res["mode"],
        })
        soc = res["soc_new"]
    return pd.DataFrame(records).set_index("timestamp")

In [ ]:
# 6-a load data
df_all = read_mumbai_csv(CFG["csv_path"])

# 6-b split future
now = pd.Timestamp.now(tz="Asia/Kolkata").floor("H")
df_future = df_all[df_all.index >= now].copy()
if df_future.empty:
    raise RuntimeError("CSV has no future records – cannot schedule.")

# 6-c run
schedule = simulate(df_future, CFG)
print("Simulation finished – first 5 rows:")
display(schedule.head())

In [ ]:
def plot_tariff(sched: pd.DataFrame, horizon: int):
    """Plot tariff for chosen horizon (hours)."""
    plt.figure(figsize=(14, 4))
    sub = sched.head(horizon)
    plt.plot(sub.index, sub["tariff"], label="Hourly tariff")
    plt.ylabel("Tariff (₹/kWh)")
    plt.title("Electricity tariff")
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_tariff(schedule, CFG["plot_hours"])

In [ ]:
def plot_battery_delivery(sched: pd.DataFrame, horizon: int):
    """Positive = discharge / deliver."""
    sub = sched.head(horizon)
    plt.figure(figsize=(14, 4))
    plt.step(sub.index, sub["battery_kw"], where="post", color="C1")
    plt.axhline(0, color="grey", lw=0.8)
    plt.ylabel("Battery power (kW)")
    plt.title("Optimal battery *delivery* power schedule (+ = discharge)")
    plt.tight_layout()
    plt.show()

plot_battery_delivery(schedule, CFG["plot_hours"])

In [ ]:
schedule.to_csv("optimal_battery_power_schedule.csv")
print("Saved →", Path("optimal_battery_power_schedule.csv").resolve())